In [1]:
import pandas as pd

C:\Users\prith\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [180]:
#Modify this at your own disgression
initial_claims = pd.read_csv("InitialClaims.csv",  index_col='DATE')
mortgage_30US = pd.read_csv("MORTGAGE30US.csv",   index_col='observation_date')
national_HPI = pd.read_excel("hpi_po_monthly_hist.xlsx", sheet_name='HPI_PO_monthly_hist', skiprows=3)
state_HPI = pd.read_excel("hpi_po_state.xlsx")
unrate = pd.read_csv("UNRATE.csv", index_col='observation_date')

In [181]:
#initial_claims[(initial_claims.index > '2023-03-20') & (initial_claims.index < '2023-04-30')]
initial_claims.index = pd.to_datetime(initial_claims.index)

claims_m = initial_claims[['ICSA']].resample('ME').mean()          # monthly avg

In [183]:
mortgage_30US.index = pd.to_datetime(mortgage_30US.index)
mort_m = mortgage_30US[['MORTGAGE30US']].resample('ME').mean().rename(columns ={ 'MORTGAGE30US' : 'mort30_m'})

In [185]:
#national_HPI
national_HPI = national_HPI.set_index('Month')
national_HPI.index = national_HPI.index.to_period("M").to_timestamp("M")
national_HPI.index = pd.to_datetime(national_HPI.index)

national_HPI_m = national_HPI[['USA\n\n(NSA)']]
national_HPI_m = national_HPI_m.rename(columns={'USA\n\n(NSA)': 'HPI_NSA'})

In [187]:
nsa_col = 'index_nsa'
sa_col = 'index_sa'
state_HPI = state_HPI[['state', 'yr', 'qtr'] + [nsa_col,sa_col]].copy()

# Map quarter to months
q_to_months = {1: [1, 2, 3], 2: [4, 5, 6], 3: [7, 8, 9], 4: [10, 11, 12]}
state_HPI["month"] = state_HPI["qtr"].map(q_to_months)

# explode one row per month
state_HPI = state_HPI.explode("month", ignore_index=True)

# build month-end dates
state_HPI["date"] = pd.to_datetime(
    dict(year=state_HPI["yr"].astype(int), month=state_HPI["month"].astype(int), day=1)
).dt.to_period("M").dt.to_timestamp("M")

# final tidy columns
state_HPI = state_HPI.rename(columns={sa_col : "state_hpi_sa", nsa_col : "state_hpi_nsa"})

state_HPI_m = state_HPI.sort_values(["state","date"]).drop(columns= ['yr', 'qtr', 'month'])
state_HPI_m = state_HPI_m.set_index('date')

In [189]:
unrate.index = pd.to_datetime(unrate.index, errors='coerce') 
unrate.index = unrate.index.to_period("M").to_timestamp("M")

In [204]:
# National monthly macro (index by date)
start_date = '1991-01-02'
end_date = '2011-12-30'

claims_m = claims_m[(claims_m.index > start_date) & (claims_m.index < end_date)]
mort_m = mort_m[(mort_m.index > start_date) & (mort_m.index < end_date)]
national_HPI_m = national_HPI_m[(national_HPI_m.index > start_date) & (national_HPI_m.index < end_date)]
unrate = unrate[(unrate.index > start_date) & (unrate.index < end_date)]
state_HPI_m = state_HPI_m[(state_HPI_m.index > start_date) & (state_HPI_m.index < end_date)]

macro_us = pd.concat([claims_m, mort_m], axis=1)
macro_us = macro_us.join(national_HPI_m, how='outer')     # us_hpi_sa, [us_hpi_nsa]
macro_us = macro_us.join(unrate, how='outer')
macro_us = macro_us.join(state_HPI_m, how='outer')

In [205]:
macro_us

,ICSA,mort30_m,HPI_NSA,UNRATE,state,state_hpi_nsa,state_hpi_sa
1991-01-31,439750.0,9.6375,100.00,6.4,AK,100.00,100.00
1991-01-31,439750.0,9.6375,100.00,6.4,AL,100.00,100.00
1991-01-31,439750.0,9.6375,100.00,6.4,AR,100.00,100.00
1991-01-31,439750.0,9.6375,100.00,6.4,AZ,100.00,100.00
1991-01-31,439750.0,9.6375,100.00,6.4,CA,100.00,100.00
...,...,...,...,...,...,...,...
2011-11-30,389250.0,3.9925,178.84,8.6,VT,203.70,206.35
2011-11-30,389250.0,3.9925,178.84,8.6,WA,200.98,200.72
2011-11-30,389250.0,3.9925,178.84,8.6,WI,193.97,193.61
2011-11-30,389250.0,3.9925,178.84,8.6,WV,182.33,181.41


In [206]:
macro_us.to_csv('aggergate_macroeconomic.csv', index=True)